In [2]:
!pip install tensorflow --upgrade

     |████████████████████████████████| 320.4 MB 22 kB/s s eta 0:00:01  |▎                               | 3.1 MB 6.4 MB/s eta 0:00:50     |█▉                              | 18.7 MB 6.4 MB/s eta 0:00:48     |████████▍                       | 83.7 MB 26.5 MB/s eta 0:00:09     |██████████████▎                 | 143.0 MB 18.7 MB/s eta 0:00:10     |███████████████                 | 150.1 MB 18.7 MB/s eta 0:00:10     |███████████████▎                | 153.1 MB 18.7 MB/s eta 0:00:09     |████████████████▋               | 166.3 MB 18.9 MB/s eta 0:00:09     |█████████████████               | 169.8 MB 18.9 MB/s eta 0:00:08     |█████████████████▏              | 172.1 MB 18.9 MB/s eta 0:00:08     |█████████████████▊              | 177.0 MB 36.2 MB/s eta 0:00:04     |███████████████████▍            | 194.1 MB 36.2 MB/s eta 0:00:04     |███████████████████████▋        | 236.0 MB 18.6 MB/s eta 0:00:05     |███████████████████████████▍    | 274.1 MB 20.3 MB/s eta 0:00:03
     |██████████████████████

In [3]:
!pip install gast --upgrade

  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
ERROR: tensorflow 2.3.0 has requirement gast==0.3.3, but you'll have gast 0.4.0 which is incompatible.
  Attempting uninstall: gast
    Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3


In [4]:
!pip install -q tf-agents
!pip install -q 'gym==0.10.11'

In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'2.1.0'

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import numpy as np

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

tf.compat.v1.enable_v2_behavior()



In [3]:
class CardGameEnv(py_environment.PyEnvironment):

    def __init__(self):
        self._action_spec = array_spec.BoundedArraySpec(
                                    shape=(), dtype=np.int32, minimum=0, maximum=1, name='action')
        self._observation_spec = array_spec.BoundedArraySpec(
                                    shape=(1,), dtype=np.int32, minimum=0, name='observation')
        self._state = 0
        self._episode_ended = False

    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec

    def _reset(self):
        self._state = 0
        self._episode_ended = False
        return ts.restart(np.array([self._state], dtype=np.int32))

    def _step(self, action):

        if self._episode_ended:
            # The last action ended the episode. Ignore the current action and start
            # a new episode.
            return self.reset()

        # Make sure episodes don't go on forever.
        if action == 1:
            self._episode_ended = True
        elif action == 0:
            new_card = np.random.randint(1, 11)
            self._state += new_card
        else:
            raise ValueError('`action` should be 0 or 1.')

        if self._episode_ended or self._state >= 21:
            reward = self._state - 21 if self._state <= 21 else -21
            return ts.termination(np.array([self._state], dtype=np.int32), reward)
        else:
            return ts.transition(np.array([self._state], dtype=np.int32), reward=0.0, discount=1.0)

In [4]:
environment = CardGameEnv()
utils.validate_py_environment(environment, episodes=5)

In [9]:
get_new_card_action = np.array(0, dtype=np.int32)
end_round_action = np.array(1, dtype=np.int32)

environment = CardGameEnv()
time_step = environment.reset()
print(time_step)
cumulative_reward = time_step.reward

for _ in range(3):
    time_step = environment.step(get_new_card_action)
    print(time_step)
    cumulative_reward += time_step.reward

time_step = environment.step(end_round_action)
print(time_step)
cumulative_reward += time_step.reward
print('Final Reward = ', cumulative_reward)

TimeStep(step_type=array(0, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([0], dtype=int32))
TimeStep(step_type=array(1, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([3], dtype=int32))
TimeStep(step_type=array(1, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([5], dtype=int32))
TimeStep(step_type=array(1, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([14], dtype=int32))
TimeStep(step_type=array(2, dtype=int32), reward=array(-7., dtype=float32), discount=array(0., dtype=float32), observation=array([14], dtype=int32))
Final Reward =  -7.0


In [10]:
env = CardGameEnv()
tf_env = tf_py_environment.TFPyEnvironment(env)

print(isinstance(tf_env, tf_environment.TFEnvironment))
print("TimeStep Specs:", tf_env.time_step_spec())
print("Action Specs:", tf_env.action_spec())

True
TimeStep Specs: TimeStep(step_type=TensorSpec(shape=(), dtype=tf.int32, name='step_type'), reward=TensorSpec(shape=(), dtype=tf.float32, name='reward'), discount=BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)), observation=BoundedTensorSpec(shape=(1,), dtype=tf.int32, name='observation', minimum=array(0, dtype=int32), maximum=array(2147483647, dtype=int32)))
Action Specs: BoundedTensorSpec(shape=(), dtype=tf.int32, name='action', minimum=array(0, dtype=int32), maximum=array(1, dtype=int32))


In [129]:
class StockMarketEnv(py_environment.PyEnvironment):
    '''
    Observation: The observation should be a (90,505) matrix
    Action: A (505) vector with probabilties from 0 1, max 10 are encoded as 1, all others are 0
    Reward: dot product of the (505,1) top 10 choices with the next (1,505) returns
    '''
    def __init__(self, X):
        self._action_spec = array_spec.BoundedArraySpec(
                                    shape=(505,1), dtype=np.float64, minimum=0, maximum=1, name='action')
        self._observation_spec = array_spec.BoundedArraySpec(
                                    shape=(90,505), dtype=np.float64, minimum=-10, maximum=10 ,name='observation')
        self._X = X
        self._state = np.array(self._X[:90], dtype=np.float64)
        self._i = 0
        self._episode_ended = False
        
#         self._step_type_spec = array_spec.BoundedArraySpec(
#                                     shape=(1,), dtype=np.int32, name='step_type')
#         self._reward_spec = array_spec.BoundedArraySpec(
#                                     shape=(1,), dtype=np.float32, name='reward')
#         self._discount_spec = array_spec.BoundedArraySpec(
#                                     shape=(1,), dtype=np.float32, name='discount')

    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec

    def _reset(self):
        self._state = np.array(self._X[:90], dtype=np.float64) ## input array
        self._i = 0
        self._episode_ended = False
        return ts.restart(self._state)

    def _step(self, action):
        '''
        Given a state array:
            - Choose the top ten stocks
            - Compute the reward by taking the dot product
            - Update the new state by taking the next timestep
            - Return the ts.transition(new_state, reward, discount=1)
        '''
        if self._episode_ended:
            # The last action ended the episode. Ignore the current action and start
            # a new episode.
            return self.reset()
        
        action_state = action.copy()
        action_state[action_state.argsort()[-10:]] = 1
        
        mask = np.ones(action_state.shape, bool)
        mask[a.argsort()[-10:]] = False
        action_state[mask] = 0
        
        reward_state = np.array(self._X[91+self._i], dtype=np.float64)
        reward = np.dot(reward_state, action_state)[0]
        
        self._i += 1
        
        if self._i + 91 >= self._X.shape[0]:
            self._episode_ended = True
        
        self._state = np.array(self._X[self._i:90+self._i], dtype=np.float64)
        
        if self._episode_ended:
            return ts.termination(np.array(self._state, dtype=np.float64), reward=np.array(reward, dtype=np.float32))
        else:
            return ts.transition(np.array(self._state, dtype=np.float64), reward=np.array(reward, dtype=np.float32), discount=1.0)

In [28]:
a = np.random.uniform(0, 1, 505)
a.argsort()[-10:]

array([131, 489, 351, 402, 455, 257,  23, 462, 262, 177])

In [29]:
b = a.copy()

In [30]:
a[a.argsort()[-10:]] = 1

In [31]:
a[a.argsort()[-10:]]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [32]:
a.argsort()[-10:]

array([489, 402, 262, 455, 351, 177, 131, 462,  23, 257])

In [33]:
mask = np.ones(a.shape, bool)
mask[a.argsort()[-10:]] = False
mask

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [34]:
a[mask] = 0

In [35]:
a

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [37]:
np.dot(b, a)

9.918081739513458

In [39]:
b[a.astype('bool')]

array([0.99278274, 0.98663844, 0.99629835, 0.99223001, 0.99552795,
       0.98949967, 0.98997428, 0.99115619, 0.99496631, 0.9890078 ])

In [51]:
X_rand = np.random.uniform(-1,1,505*180).reshape(180,505)

In [52]:
import pandas as pd
pd.DataFrame(X_rand)

,0,1,2,3,4,5,6,7,8,9,...,495,496,497,498,499,500,501,502,503,504
0,0.140540,-0.329190,-0.047001,-0.649297,-0.633723,-0.838608,-0.111109,0.287581,0.193419,-0.045589,...,-0.550927,-0.231102,-0.839100,0.961155,0.646570,0.985204,0.674152,-0.357929,0.743289,-0.471797
1,-0.423886,0.695772,-0.165877,0.484184,0.670172,0.602067,-0.240509,-0.035103,-0.795709,0.185046,...,-0.766458,0.514237,0.406589,0.112033,0.584977,-0.668325,0.783343,0.435287,-0.384862,-0.760927
2,0.085121,0.102740,0.279178,-0.211539,0.310252,0.602915,0.491089,0.117420,0.416565,-0.542193,...,0.975062,0.616642,-0.756611,0.566252,-0.272756,-0.419373,0.357208,-0.607496,0.123153,-0.545897
3,-0.462133,-0.130015,-0.727195,-0.464527,-0.623254,-0.883614,-0.459878,-0.896452,0.614098,0.104455,...,0.870758,-0.863817,0.274780,0.260244,-0.121842,0.257807,0.139573,0.194738,0.149159,0.266315
4,0.558112,-0.058520,0.686742,-0.643676,0.658531,0.869654,-0.751750,-0.127664,-0.764127,-0.482681,...,0.705809,0.099999,-0.990344,0.063195,0.090226,-0.552996,0.524431,-0.628807,0.512478,-0.272679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,0.714755,0.842858,0.238492,0.377687,-0.245636,0.449620,0.451549,0.368687,-0.606766,0.817496,...,-0.694301,-0.411061,-0.264308,-0.254222,-0.765083,0.482651,0.375840,-0.251152,0.616515,0.400360
176,-0.326593,-0.890037,0.153252,-0.295029,-0.820342,0.412458,0.612178,-0.113631,-0.471429,0.732446,...,-0.394110,-0.034896,-0.054923,0.394835,0.273408,-0.727077,-0.024420,0.890272,0.251708,-0.729621
177,-0.319389,0.569504,-0.741532,-0.209200,0.316878,0.939107,-0.937982,-0.294802,-0.588976,-0.755803,...,-0.350568,0.078743,0.119772,-0.063003,0.188037,-0.990717,-0.876366,0.811248,0.610377,-0.891328
178,0.978380,0.322295,-0.667739,0.436656,-0.056940,-0.654141,0.367164,-0.166018,-0.626665,-0.402560,...,-0.958491,-0.261104,-0.635241,0.564070,0.786827,-0.714524,0.319717,-0.950689,-0.713407,-0.967913


In [130]:
environment = StockMarketEnv(X_rand)
utils.validate_py_environment(environment, episodes=5)